In [1]:
# !pip3 install pandas numpy torch transformers scikit-learn
# !pip3 install sentence-transformers

In [2]:
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import re

/mnt/UDev/train_dataset_tag_video/venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


# Обработка текстовых данных

In [4]:
def preprocess_text(text):
    if not isinstance(text, str):
        return ''
    # Удаление лишних символов
    text = text.replace('\n', ' ').replace('\r', ' ')
    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text)
    # Опционально: приведение к нижнему регистру (если используете uncased модель)
    # text = text.lower()
    return text.strip()

In [5]:
video_df = pd.read_csv('train_data_categories.csv')
video_df.head()

,video_id,title,description,tags
0,9007f33c8347924ffa12f922da2a179d,Пацанский клининг. Шоу «ЧистоТачка» | Повелите...,Тяпа и Егор бросили вызов нестареющему «повели...,Массовая культура: Юмор и сатира
1,9012707c45233bd601dead57bc9e2eca,"СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...","CarJitsu — бои в формате POP MMA, где вместо р...",События и достопримечательности: Спортивные с...
2,e01d6ebabbc27e323fa1b7c581e9b96a,"Злые языки | Выпуск 1, Сезон 1 | Непорочность ...",Почему Дана Борисова предпочитает молчать о по...,"Массовая культура: Отношения знаменитостей, Ма..."
3,a00b145242be3ebc3b311455e94917af,$1000 шоу | 1 выпуск | Автобоулинг,"В этом выпуске, популярный автоблогер Дима Гор...","Транспорт, Спорт: Автогонки, Массовая культура"
4,b01a682bf4dfcc09f1e8fac5bc18785a,В РОТ МНЕ НОТЫ #1 ВИТА ЧИКОВАНИ,В первом выпуске «В рот мне ноты» популярная п...,Массовая культура: Юмор и сатира


In [7]:
video_df['title_clean'] = video_df['title'].apply(preprocess_text)
video_df['description_clean'] = video_df['description'].apply(preprocess_text)
video_df.head()

,video_id,title,description,tags,title_clean,description_clean
0,9007f33c8347924ffa12f922da2a179d,Пацанский клининг. Шоу «ЧистоТачка» | Повелите...,Тяпа и Егор бросили вызов нестареющему «повели...,Массовая культура: Юмор и сатира,Пацанский клининг. Шоу «ЧистоТачка» | Повелите...,Тяпа и Егор бросили вызов нестареющему «повели...
1,9012707c45233bd601dead57bc9e2eca,"СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...","CarJitsu — бои в формате POP MMA, где вместо р...",События и достопримечательности: Спортивные с...,"СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...","CarJitsu — бои в формате POP MMA, где вместо р..."
2,e01d6ebabbc27e323fa1b7c581e9b96a,"Злые языки | Выпуск 1, Сезон 1 | Непорочность ...",Почему Дана Борисова предпочитает молчать о по...,"Массовая культура: Отношения знаменитостей, Ма...","Злые языки | Выпуск 1, Сезон 1 | Непорочность ...",Почему Дана Борисова предпочитает молчать о по...
3,a00b145242be3ebc3b311455e94917af,$1000 шоу | 1 выпуск | Автобоулинг,"В этом выпуске, популярный автоблогер Дима Гор...","Транспорт, Спорт: Автогонки, Массовая культура",$1000 шоу | 1 выпуск | Автобоулинг,"В этом выпуске, популярный автоблогер Дима Гор..."
4,b01a682bf4dfcc09f1e8fac5bc18785a,В РОТ МНЕ НОТЫ #1 ВИТА ЧИКОВАНИ,В первом выпуске «В рот мне ноты» популярная п...,Массовая культура: Юмор и сатира,В РОТ МНЕ НОТЫ #1 ВИТА ЧИКОВАНИ,В первом выпуске «В рот мне ноты» популярная п...


In [8]:
rus_df = pd.read_json('rus.json').T
rus_df.head()

,audio,video
00efa58930724f2ae6f9916f53cda3b3,"{'0': ' Деньги в онлайн-магазинах', '1': ' Муж...",1. Self-care\n2. Mental health\n3. Stress mana...
01bd6b00e670655ca7ff484ff55d8ad9,"{'0': ' рождения точно. Да, в вашем случае, Се...",1. Physical Fitness\n2. Strength Training\n3. ...
01dcabe37935dbbffd389c22c327f361,"{'0': ' Субтитры подогнал «Симон» Друзья, добр...",1. Cartoon\n2. Humor\n3. Animation\n4. Comedy\...
02b797508b9ffc941887703dfb17d365,"{'0': ' Я сказал, смотрите на меня, я же выгля...","Sure, I'd be happy to help! Here are some tags..."
02d4cc029c5a7531f36992446a24478f,{'0': ' здесь сабли написано я могу сказать чт...,"Sure, I'd be happy to help! Here are some tags..."


In [9]:
def process_audio(audio):
    if isinstance(audio, dict):
        return ' '.join(audio.values())
    elif isinstance(audio, str):
        return audio
    else:
        return ''

In [10]:
import re

def extract_tags_from_description(description):
    if not isinstance(description, str):
        return ''
    
    description = description.strip().replace('\n', ' ')
    pattern = r'\d+\.\s*(.*?)\s*(?=\d+\.|$)'
    matches = re.findall(pattern, description)
    tags = [match.strip().strip('.').strip(',') for match in matches]
    tags_text = ' '.join(tags)
    
    return tags_text



rus_df['video_clean'] = rus_df['video'].apply(extract_tags_from_description)
rus_df['video_clean'] = rus_df['video_clean'].apply(preprocess_text)
rus_df['video_clean'].head()

00efa58930724f2ae6f9916f53cda3b3    Self-care Mental health Stress management Pers...
01bd6b00e670655ca7ff484ff55d8ad9    Physical Fitness Strength Training Fitness Mot...
01dcabe37935dbbffd389c22c327f361       Cartoon Humor Animation Comedy Family-friendly
02b797508b9ffc941887703dfb17d365    Health and wellness Self-care Stress managemen...
02d4cc029c5a7531f36992446a24478f    Time management Productivity Work-life balance...
Name: video_clean, dtype: object

In [14]:
rus_df['audio_clean'] = rus_df['audio'].apply(process_audio)
rus_df['audio_clean'] = rus_df['audio_clean'].apply(preprocess_text)
rus_df['audio_clean'].head()

# set id column as new column
rus_df = rus_df.reset_index()
rus_df = rus_df.rename(columns={'index': 'video_id'})

In [15]:
rus_df.head()

,level_0,video_id,audio,video,video_clean,audio_clean
0,0,00efa58930724f2ae6f9916f53cda3b3,"{'0': ' Деньги в онлайн-магазинах', '1': ' Муж...",1. Self-care\n2. Mental health\n3. Stress mana...,Self-care Mental health Stress management Pers...,Деньги в онлайн-магазинах Мужчины Ты еще молот...
1,1,01bd6b00e670655ca7ff484ff55d8ad9,"{'0': ' рождения точно. Да, в вашем случае, Се...",1. Physical Fitness\n2. Strength Training\n3. ...,Physical Fitness Strength Training Fitness Mot...,"рождения точно. Да, в вашем случае, Сергей, им..."
2,2,01dcabe37935dbbffd389c22c327f361,"{'0': ' Субтитры подогнал «Симон» Друзья, добр...",1. Cartoon\n2. Humor\n3. Animation\n4. Comedy\...,Cartoon Humor Animation Comedy Family-friendly,"Субтитры подогнал «Симон» Друзья, добрый вечер..."
3,3,02b797508b9ffc941887703dfb17d365,"{'0': ' Я сказал, смотрите на меня, я же выгля...","Sure, I'd be happy to help! Here are some tags...",Health and wellness Self-care Stress managemen...,"Я сказал, смотрите на меня, я же выгляжу не ка..."
4,4,02d4cc029c5a7531f36992446a24478f,{'0': ' здесь сабли написано я могу сказать чт...,"Sure, I'd be happy to help! Here are some tags...",Time management Productivity Work-life balance...,здесь сабли написано я могу сказать что вот са...


In [16]:
df = video_df.copy()
df = pd.merge(video_df, rus_df, on='video_id', how='left')

# Проверим размеры
print(f"Количество видео после объединения: {len(df)}") 

Количество видео после объединения: 1049


In [17]:
df_clean = df[['video_id', 'title_clean', 'description_clean', 'video_clean', 'audio_clean']]
df_clean.head()

,video_id,title_clean,description_clean,video_clean,audio_clean
0,9007f33c8347924ffa12f922da2a179d,Пацанский клининг. Шоу «ЧистоТачка» | Повелите...,Тяпа и Егор бросили вызов нестареющему «повели...,NaN,NaN
1,9012707c45233bd601dead57bc9e2eca,"СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...","CarJitsu — бои в формате POP MMA, где вместо р...",NaN,NaN
2,e01d6ebabbc27e323fa1b7c581e9b96a,"Злые языки | Выпуск 1, Сезон 1 | Непорочность ...",Почему Дана Борисова предпочитает молчать о по...,NaN,NaN
3,a00b145242be3ebc3b311455e94917af,$1000 шоу | 1 выпуск | Автобоулинг,"В этом выпуске, популярный автоблогер Дима Гор...",NaN,NaN
4,b01a682bf4dfcc09f1e8fac5bc18785a,В РОТ МНЕ НОТЫ #1 ВИТА ЧИКОВАНИ,В первом выпуске «В рот мне ноты» популярная п...,NaN,NaN


In [19]:
df_clean = df_clean.fillna('')
df_clean.isna().sum()

video_id             0
title_clean          0
description_clean    0
video_clean          0
audio_clean          0
dtype: int64

# Загрузка тегов

In [24]:
tags_df = pd.read_csv('IAB_tags.csv')  # Замените на ваш файл
tags_df = tags_df[['Уровень 1 (iab)', 'Уровень 2 (iab)', 'Уровень 3 (iab)']].fillna('')

# Объединение уровней тегов
def combine_tag_levels(row):
    tags = [row['Уровень 1 (iab)'], row['Уровень 2 (iab)'], row['Уровень 3 (iab)']]
    tags = [tag.strip() for tag in tags if tag.strip()]
    return ': '.join(tags)

tags_df['full_tag'] = tags_df.apply(combine_tag_levels, axis=1)

In [21]:
from transformers import AutoTokenizer, AutoModel

# Установка устройства
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Используемое устройство: {device}')

model_name = 'paraphrase-multilingual-mpnet-base-v2'
model = SentenceTransformer(model_name, device=device)

Используемое устройство: cuda


/mnt/UDev/train_dataset_tag_video/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Case 1



In [26]:
df_clean['combined_text'] = df_clean['title_clean'] + ' ' + df_clean['description_clean']
df_clean['embedding'] = df_clean['combined_text'].apply(lambda x: model.encode(x, convert_to_tensor=True))
tags_df['embedding'] = tags_df['full_tag'].apply(lambda x: model.encode(x, convert_to_tensor=True))



In [32]:
tag_embeddings = torch.stack(tags_df['embedding'].tolist())
tag_list = tags_df['full_tag'].tolist()

def assign_tags(video_embedding, tag_embeddings, tags, top_k=4):
    similarities = cosine_similarity(video_embedding.cpu().numpy(), tag_embeddings.cpu().numpy())[0]
    top_indices = similarities.argsort()[-top_k:][::-1]
    assigned_tags = [tags[i] for i in top_indices]
    return assigned_tags

df['assigned_tags'] = df_clean['embedding'].apply(lambda emb: assign_tags(emb.unsqueeze(0), tag_embeddings, tag_list, top_k=2))

# Просмотр результатов
print(df[['video_id', 'assigned_tags', 'tags']].head())

                           video_id  \
0  9007f33c8347924ffa12f922da2a179d   
1  9012707c45233bd601dead57bc9e2eca   
2  e01d6ebabbc27e323fa1b7c581e9b96a   
3  a00b145242be3ebc3b311455e94917af   
4  b01a682bf4dfcc09f1e8fac5bc18785a   

                                       assigned_tags  \
0         [Телевидение: Драма, Телевидение: Сериалы]   
1           [Спорт: Боевые искусства, Спорт: Борьба]   
2  [Фильмы и анимация: Драмы, Телевидение: Реалит...   
3  [Покупки: Лотереи и скретч-карты, Спорт: Автог...   
4  [Телевидение: Комедийные каналы, Музыка и ауди...   

                                                tags  
0                   Массовая культура: Юмор и сатира  
1  События и достопримечательности:  Спортивные с...  
2  Массовая культура: Отношения знаменитостей, Ма...  
3     Транспорт, Спорт: Автогонки, Массовая культура  
4                   Массовая культура: Юмор и сатира  


In [33]:
def calculate_iou(predicted_tags, real_tags):
    predicted_set = set(predicted_tags)
    real_set = set(real_tags)
    intersection = predicted_set.intersection(real_set)
    union = predicted_set.union(real_set)
    if not union:
        return 0.0
    iou = len(intersection) / len(union)
    return iou

In [34]:
df = df.fillna('')
df['iou'] = df.apply(lambda row: calculate_iou(row['assigned_tags'], row['tags']), axis=1)
mean_iou = df['iou'].mean()
print(f"Средний IoU: {mean_iou:.4f}")
max_iou = df['iou'].max()
print(f"Максимальный IoU: {max_iou:.4f}")
min_iou = df['iou'].min()
print(f"Минимальный IoU: {min_iou:.4f}")

Средний IoU: 0.0000
Максимальный IoU: 0.0000
Минимальный IoU: 0.0000


# Case 2

In [37]:
# Получение уникальных тегов 1 уровня
level1_tags = tags_df['Уровень 1 (iab)'].unique()
level1_tags = [tag for tag in level1_tags if tag]

# Подготовка данных тегов 1 уровня
level1_df = pd.DataFrame({'tag': level1_tags})
level1_df['tag_clean'] = level1_df['tag'].apply(preprocess_text)
level1_df['embedding'] = level1_df['tag_clean'].apply(lambda x: model.encode(x, convert_to_tensor=True))

level1_embeddings = torch.stack(level1_df['embedding'].tolist())
level1_tag_list = level1_df['tag'].tolist()

def assign_level_tags(video_embedding, tag_embeddings, tags, top_k=1):
    similarities = cosine_similarity(video_embedding.cpu().numpy(), tag_embeddings.cpu().numpy())[0]
    top_indices = similarities.argsort()[-top_k:][::-1]
    assigned_tags = [tags[i] for i in top_indices]
    return assigned_tags

df['level1_tags'] = df_clean['embedding'].apply(lambda emb: assign_level_tags(emb.unsqueeze(0), level1_embeddings, level1_tag_list, top_k=3))


In [38]:
df[['video_id', 'level1_tags', 'tags']].head()

,video_id,level1_tags,tags
0,9007f33c8347924ffa12f922da2a179d,"[Карьера, Телевидение, Медицина]",Массовая культура: Юмор и сатира
1,9012707c45233bd601dead57bc9e2eca,"[Фильмы и анимация, Игры, Телевидение]",События и достопримечательности: Спортивные с...
2,e01d6ebabbc27e323fa1b7c581e9b96a,"[Карьера, Фильмы и анимация, Новости и политика]","Массовая культура: Отношения знаменитостей, Ма..."
3,a00b145242be3ebc3b311455e94917af,"[Игры, Покупки, Карьера]","Транспорт, Спорт: Автогонки, Массовая культура"
4,b01a682bf4dfcc09f1e8fac5bc18785a,"[Карьера, Телевидение, Фильмы и анимация]",Массовая культура: Юмор и сатира


# case 3

In [39]:
df_clean

,video_id,title_clean,description_clean,video_clean,audio_clean,combined_text,embedding
0,9007f33c8347924ffa12f922da2a179d,Пацанский клининг. Шоу «ЧистоТачка» | Повелите...,Тяпа и Егор бросили вызов нестареющему «повели...,,,Пацанский клининг. Шоу «ЧистоТачка» | Повелите...,"[tensor(0.0961, device='cuda:0'), tensor(0.196..."
1,9012707c45233bd601dead57bc9e2eca,"СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...","CarJitsu — бои в формате POP MMA, где вместо р...",,,"СarJitsu. 3 сезон, 6 серия. Нарек Симонян vs Ж...","[tensor(0.1175, device='cuda:0'), tensor(0.159..."
2,e01d6ebabbc27e323fa1b7c581e9b96a,"Злые языки | Выпуск 1, Сезон 1 | Непорочность ...",Почему Дана Борисова предпочитает молчать о по...,,,"Злые языки | Выпуск 1, Сезон 1 | Непорочность ...","[tensor(-0.0022, device='cuda:0'), tensor(0.27..."
3,a00b145242be3ebc3b311455e94917af,$1000 шоу | 1 выпуск | Автобоулинг,"В этом выпуске, популярный автоблогер Дима Гор...",,,$1000 шоу | 1 выпуск | Автобоулинг В этом выпу...,"[tensor(0.0446, device='cuda:0'), tensor(0.052..."
4,b01a682bf4dfcc09f1e8fac5bc18785a,В РОТ МНЕ НОТЫ #1 ВИТА ЧИКОВАНИ,В первом выпуске «В рот мне ноты» популярная п...,,,В РОТ МНЕ НОТЫ #1 ВИТА ЧИКОВАНИ В первом выпус...,"[tensor(0.0486, device='cuda:0'), tensor(0.119..."
...,...,...,...,...,...,...,...
1044,5fe16aa2869667bc1519e32a4c536b26,"Злые языки | Выпуск 3, Сезон 1 | Эксклюзив Над...",Гость выпуска – Надин Серовски. Ей предстоит о...,,,"Злые языки | Выпуск 3, Сезон 1 | Эксклюзив Над...","[tensor(0.0433, device='cuda:0'), tensor(0.245..."
1045,4ffa5fbb2a410aa841659d8890ae5e3f,МАКСИМ НАРОДНЫЙ Выпуск №15 ГОТОВИМ «ПОХМЕЛЬНЫЙ...,Предлагаю подписчикам быстро приготовить похме...,,,МАКСИМ НАРОДНЫЙ Выпуск №15 ГОТОВИМ «ПОХМЕЛЬНЫЙ...,"[tensor(-0.0237, device='cuda:0'), tensor(0.06..."
1046,3fc81df4bfe121ce2bc33dd581f5efeb,Роман Юнусов и блогерка Арина Ростовская пытаю...,В новом выпуске шоу «Спортивный Интерес» Рома ...,Fencing Techniques Fencing Training Fencing Eq...,вообще куда пришел это все бесплатно бесплатно...,Роман Юнусов и блогерка Арина Ростовская пытаю...,"[tensor(0.0921, device='cuda:0'), tensor(0.048..."
1047,efe0b4139ef82ec270b9e2fe0216214e,Артмеханика. Сезон 2. Выпуск 18. Современные р...,“Артмеханика” представляет своих друзей! Поэты...,,,Артмеханика. Сезон 2. Выпуск 18. Современные р...,"[tensor(0.0443, device='cuda:0'), tensor(0.132..."


In [41]:
# Получение эмбеддингов
df_clean['audio_embedding'] = df_clean['audio_clean'].apply(lambda x: model.encode(x, convert_to_tensor=True))
df_clean['video_embedding'] = df_clean['video_clean'].apply(lambda x: model.encode(x, convert_to_tensor=True))
df_clean['title_embedding'] = df_clean['title_clean'].apply(lambda x: model.encode(x, convert_to_tensor=True))
df_clean['description_embedding'] = df_clean['description_clean'].apply(lambda x: model.encode(x, convert_to_tensor=True))


In [73]:
# Задание весов
weights = {
    'audio': 1,
    'description': 1,
    'video_tags': 1,
    'title': 1
}

def combine_embeddings(row):
    combined_embedding = (
        row['audio_embedding'].cpu().numpy() * weights['audio'] +
        row['description_embedding'].cpu().numpy() * weights['description'] +
        row['video_embedding'].cpu().numpy() * weights['video_tags'] +
        row['title_embedding'].cpu().numpy() * weights['title']
    )
    return combined_embedding

df_clean['combined_embedding'] = df_clean.apply(combine_embeddings, axis=1)


In [74]:
tags_df.head()

,Уровень 1 (iab),Уровень 2 (iab),Уровень 3 (iab),full_tag,embedding,tag_embedding
0,Транспорт,,,Транспорт,"[tensor(-0.0193, device='cuda:0'), tensor(0.15...","[tensor(-0.0193), tensor(0.1599), tensor(-0.01..."
1,Транспорт,Типы кузова автомобиля,,Транспорт: Типы кузова автомобиля,"[tensor(0.0236, device='cuda:0'), tensor(0.021...","[tensor(0.0236), tensor(0.0214), tensor(-0.013..."
2,Транспорт,Типы кузова автомобиля,Грузовой автомобиль,Транспорт: Типы кузова автомобиля: Грузовой ав...,"[tensor(0.0183, device='cuda:0'), tensor(0.022...","[tensor(0.0183), tensor(0.0227), tensor(-0.013..."
3,Транспорт,Типы кузова автомобиля,Седан,Транспорт: Типы кузова автомобиля: Седан,"[tensor(-0.0125, device='cuda:0'), tensor(-0.0...","[tensor(-0.0125), tensor(-0.0319), tensor(-0.0..."
4,Транспорт,Типы кузова автомобиля,Универсал,Транспорт: Типы кузова автомобиля: Универсал,"[tensor(0.0143, device='cuda:0'), tensor(0.026...","[tensor(0.0143), tensor(0.0267), tensor(-0.013..."


In [75]:
tags_df['tag_embedding'] = tags_df['full_tag'].apply(lambda x: model.encode(x, convert_to_tensor=True).cpu())

In [76]:
tags_df = tags_df[tags_df['full_tag'] != '']

In [80]:
df_clean['full_text'] = df_clean['title_clean'] + ' ' + df_clean['description_clean'] + ' ' + df_clean['video_clean'] + ' ' + df_clean['audio_clean']
df_clean['full_text'] = df_clean['full_text'].apply(preprocess_text)
df_clean['full_text_embedding'] = df_clean['full_text'].apply(lambda x: model.encode(x, convert_to_tensor=True).cpu())


In [85]:
# Преобразование эмбеддингов тегов в массив NumPy
tag_embeddings = np.vstack(tags_df['tag_embedding'].values)

# Функция для присвоения тегов видео
def assign_tags(combined_embedding):
    similarities = cosine_similarity([combined_embedding], tag_embeddings)[0]
    tag_similarities = list(zip(tags_df['full_tag'], similarities))
    tag_similarities = sorted(tag_similarities, key=lambda x: x[1], reverse=True)
    top_n = 5
    top_tags = [tag for tag, sim in tag_similarities[:top_n]]
    return top_tags

# Присвоение тегов каждому видео
df_clean['predicted_tags'] = df_clean['full_text_embedding'].apply(assign_tags)

# Просмотр результатов
df_clean[['video_id', 'predicted_tags']]


,video_id,predicted_tags
0,9007f33c8347924ffa12f922da2a179d,"[Телевидение: Драма, Телевидение: Сериалы, Изо..."
1,9012707c45233bd601dead57bc9e2eca,"[Спорт: Боевые искусства, Спорт: Борьба, Спорт..."
2,e01d6ebabbc27e323fa1b7c581e9b96a,"[Фильмы и анимация: Драмы, Телевидение: Реалит..."
3,a00b145242be3ebc3b311455e94917af,"[Покупки: Лотереи и скретч-карты, Спорт: Автог..."
4,b01a682bf4dfcc09f1e8fac5bc18785a,"[Телевидение: Комедийные каналы, Музыка и ауди..."
...,...,...
1044,5fe16aa2869667bc1519e32a4c536b26,"[Телевидение: Реалити-ТВ, Карьера, Телевидение..."
1045,4ffa5fbb2a410aa841659d8890ae5e3f,"[Еда и напитки: Кулинария, Еда и напитки: Рест..."
1046,3fc81df4bfe121ce2bc33dd581f5efeb,"[Телевидение: Реалити-ТВ, Спорт: Экстремальные..."
1047,efe0b4139ef82ec270b9e2fe0216214e,[Музыка и аудио: Комедия и стендап (Музыка и а...


In [86]:
# Если у вас есть реальные теги для сравнения, можно вычислить метрику IoU

# Преобразование реальных тегов в списки
df_clean['real_tags_list'] = df['tags'].apply(lambda x: [tag.strip() for tag in x.split(',')])

# Функция для вычисления IoU
def calculate_iou(predicted, real):
    predicted_set = set(predicted)
    real_set = set(real)
    intersection = predicted_set.intersection(real_set)
    union = predicted_set.union(real_set)
    if not union:
        return 0.0
    return len(intersection) / len(union)

# Вычисление IoU для каждого видео
df_clean['iou'] = df_clean.apply(lambda row: calculate_iou(row['predicted_tags'], row['real_tags_list']), axis=1)

# Среднее значение IoU
mean_iou = df['iou'].mean()
print(f'Средний IoU по всем видео: {mean_iou:.4f}')


Средний IoU по всем видео: 0.0000


In [83]:
df['iou'].max()

0.0

# Case 4

In [87]:
from transformers import AutoTokenizer, AutoModel

# Установка устройства
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Используемое устройство: {device}')

# Загрузка токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased')
model.to(device)
model.eval()


Используемое устройство: cuda


Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [88]:
def get_bert_embedding(text):
    if not text.strip():
        # Если текст пустой, возвращаем нулевой вектор
        return np.zeros(model.config.hidden_size)
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512, padding='max_length')
    inputs = {key: value.to(device) for key, value in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    # Используем эмбеддинг [CLS] токена
    embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return embedding.flatten()


In [90]:
# Получение эмбеддингов для названия
df_clean['title_embedding'] = df_clean['title_clean'].apply(get_bert_embedding)
# Получение эмбеддингов для описания
df_clean['description_embedding'] = df_clean['description_clean'].apply(get_bert_embedding)
# Получение эмбеддингов для видео
df_clean['video_embedding'] = df_clean['video_clean'].apply(get_bert_embedding)
# Получение эмбеддингов для аудио
df_clean['audio_embedding'] = df_clean['audio_clean'].apply(get_bert_embedding)



In [96]:
def combine_embeddings(row):
    embeddings = []
    weights = []
    
    # Название
    if not np.all(row['title_embedding'] == 0):
        embeddings.append(row['title_embedding'])
        weights.append(0.1)
    
    # Описание
    if not np.all(row['description_embedding'] == 0):
        embeddings.append(row['description_embedding'])
        weights.append(0.3)
    
    # Аудио
    if not np.all(row['audio_embedding'] == 0):
        embeddings.append(row['audio_embedding'])
        weights.append(0.4)
    
    # Видео текст
    if not np.all(row['video_embedding'] == 0):
        embeddings.append(row['video_embedding'])
        weights.append(0.2)
    
    if not embeddings:
        # Если нет доступных эмбеддингов, возвращаем нулевой вектор
        return np.zeros(model.config.hidden_size)
    
    # Нормализуем веса
    total_weight = sum(weights)
    weights = [w / total_weight for w in weights]
    
    # Вычисляем взвешенное среднее эмбеддингов
    combined_embedding = np.average(embeddings, axis=0, weights=weights)
    return combined_embedding


In [97]:
df_clean['combined_embedding'] = df_clean.apply(combine_embeddings, axis=1)


In [103]:
def assign_tags(combined_embedding):
    if np.all(combined_embedding == 0):
        # Если эмбеддинг пустой, возвращаем пустой список
        return []
    similarities = cosine_similarity([combined_embedding], tag_embeddings)[0]
    tag_similarities = list(zip(tags_df['full_tag'], similarities))
    tag_similarities = sorted(tag_similarities, key=lambda x: x[1], reverse=False)
    top_n = 5
    top_tags = [tag for tag, sim in tag_similarities[:top_n]]
    return top_tags

# Применение функции к каждому видео
df_clean['predicted_tags'] = df_clean['combined_embedding'].apply(assign_tags)
df_clean['tags'] = df['tags']

In [104]:
df_clean[['video_id', 'predicted_tags', 'tags']]

,video_id,predicted_tags,tags
0,9007f33c8347924ffa12f922da2a179d,"[Семья и отношения: Смерть родственников, Дом ...",Массовая культура: Юмор и сатира
1,9012707c45233bd601dead57bc9e2eca,"[Религия и духовность: Духовность, Религия и д...",События и достопримечательности: Спортивные с...
2,e01d6ebabbc27e323fa1b7c581e9b96a,"[Дом и сад: Системы безопасности, Семья и отно...","Массовая культура: Отношения знаменитостей, Ма..."
3,a00b145242be3ebc3b311455e94917af,"[Дом и сад, Религия и духовность: Буддизм, Дом...","Транспорт, Спорт: Автогонки, Массовая культура"
4,b01a682bf4dfcc09f1e8fac5bc18785a,"[Религия и духовность: Духовность, Медицина: М...",Массовая культура: Юмор и сатира
...,...,...,...
1044,5fe16aa2869667bc1519e32a4c536b26,"[Семья и отношения: Смерть родственников, Спор...",Массовая культура:Отношения знаменитостей: Сем...
1045,4ffa5fbb2a410aa841659d8890ae5e3f,"[Дом и сад, Дом и сад: Наружная отделка, Дом и...",Еда и напитки: Кулинария
1046,3fc81df4bfe121ce2bc33dd581f5efeb,"[Семья и отношения: Смерть родственников, Спор...",Спорт
1047,efe0b4139ef82ec270b9e2fe0216214e,"[Религия и духовность: Агностицизм, Религия и ...","Книги и литература: Поэзия, Музыка и аудио, К..."
